In [1]:
import time
import cv2
import mujoco as mj
import mujoco.viewer
import mujoco.glfw

import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial import ConvexHull
from utility import unit
from controller import Controller

from human import Human
import time
import mujoco

import mujoco.renderer
import mujoco.glfw
import numpy as np
import open3d as o3d
import math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
from sensors import Sensors

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def move_simple(name, *, t, func, axis=-1, model):
  model.site(name).pos[axis] = func(t)

def ConvexHull_points(mat):
  print("ConvexHull")
  points_hull = mat[:,0:3]

  hull = ConvexHull(points_hull)
  centroid = np.mean(points_hull[hull.vertices, :], axis=0)

  x = np.append(mat[:, 0], centroid[0])
  y = np.append(mat[:, 1], centroid[1])
  z = np.append(mat[:, 2], centroid[2])
  print(centroid[0])
  print(np.mean(x))
  print(centroid[1])
  print(np.mean(y))
  print(centroid[2])
  print(np.mean(z))
  
  return x, y, z



In [3]:
m = mj.MjModel.from_xml_path('./mjcf/tec.xml')
d = mj.MjData(m)
r = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
ren_orig = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
ren_dep = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
ren_seg = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
opt = mujoco.MjvOption()
opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True
sensor = Sensors()


frame = 0

cam = mujoco.MjvCamera()
result = {}
poses = []
rotations = []
points = []

point = []
point_cloud = []

t = 0
start_sim = d.time
no = -1
z_angle = 0
with mujoco.viewer.launch_passive(m, d) as viewer:
    # Close the viewer automatically after 30 wall-seconds.
    with viewer.lock():

      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)
    viewer.sync()
    


    start = time.time()
    while viewer.is_running() and time.time() - start < 2:
      step_start = time.time()
    
    
      # policy
      # mj_step can be replaced with code that also evaluates
      # a policy and applies a contro l signal before stepping the physics.

      #move_simple('human1', t=t, func=lambda x: (np.sin(x)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      #move_simple('human2', t=t, func=lambda x: (np.sin(x*2)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      #move_simple('human3', t=t, func=lambda x: (np.sin(x*3)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      mujoco.mj_step(m, d)
      viewer.sync()

      t += 0.1
      

      
      
      
  
      # Pick up changes to the physics state, apply perturbations, update options from GUI.
      ren_dep.enable_depth_rendering()
      ren_seg.enable_segmentation_rendering()
      readings = sensor.get_rgbd_seg_matrices(m, d, ren_orig, ren_dep, ren_seg)
      print(readings)
      #print(range(m.cam_user.shape[0]))

      

      #geom_ids = seg[:, :, 0]
      #print(geom_ids)

      #print(geom_ids.shape)
      
      
      #depth_r = r.render()

      #print(depth_r2)
      #depth = np.expand_dims(depth_r, axis=2)
      #depth = np.expand_dims(depth_r, axis=2)
      #depth[depth < 0.01] = np.nan
      #depth[depth > 4.0] = np.nan
      #print(depth.shape)
      #print(depth_r.shape)
      #print(depth)
      
      #print(result[no].shape)

      if d.time - start_sim > 0.092:
    
        frame = frame +1
        start_sim = d.time

      # Rudimentary time keeping, will drift relative to wall clock.
      time_until_next_step = m.opt.timestep - (time.time() - step_start)
      if time_until_next_step > 0:
        time.sleep(time_until_next_step)
      time.sleep(0.01)
    ren_dep.disable_depth_rendering()
    ren_seg.disable_segmentation_rendering()
    

{0: array([[[ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        ...,
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00]],

       [[ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        [ 1.80000000e+02,  1.80000000e+02,  1.80000000e+02,
                     nan, -1.00000000e+00],
        ...,
        [ 1.80000000e+02,  1.800

In [4]:
frames = [readings[i][:, :, 3] for i in range(len(readings))]
segments = [readings[i][:, :, 4] for i in range(len(readings))]

poses = []
rotations = []
for i in range(m.cam_user.shape[0]):
# Get the position of the camera
  pos = m.cam_pos[i] + m.body(m.cam_bodyid[i]).pos
  poses.append(pos)

  # Get the rotation of the camera
  angle = R.from_quat(m.body(m.cam_bodyid[i]).quat).as_euler('xyz', degrees=True)
  rotations.append(angle[0])


In [5]:
import point_cloud as pcu

point_cloud = pcu.PointCloud((480, 640), 60, downsample=2)
mat = point_cloud.get_map(frames, poses, rotations)


segemented_mat = point_cloud.get_segmented_map(frames, segments, poses, rotations, m)
#mat_avg = point_cloud.get_segmented_weigavg_map(frames, segments, poses, rotations, m)


#for i in range(3):
#for j in range(len(mat_avg)):
#feature_vector = point_cloud.get_feature_vector(key, np.array(mat_avg[i][j]))

#print(len(mat_avg))

feature_vectors = point_cloud.get_feature_vectors(segmented_map=segemented_mat)
#print(feature_vectors_avg)
#feat_vec_mat = point_cloud.get_feature_avgweig_vectors(m = m,segmented_map=mat_avg)

x, y, z = ConvexHull_points(mat)





ConvexHull
0.1787033847831662
0.30649528222208683
0.012781875366792218
0.03153283918018431
0.0955363079976054
0.09880574207424585


In [14]:
"""
Returns a set of feature vectors for each agent in the map

Parameters
----------
frames : A list of depth frames from the camera.
segments : A list of segmentation frames from the camera.
poses : A list of camera poses in the world frame.
rotations : A list of camera rotations in the world frame.
m : The mujoco model.
"""

# Creating a list of points in the world frame
segmented_point_cloud = {}
segmented_pc_f = {}
segmented_point_cloudlast = {}
point_individual = {}
point_individual_pc = {}
yes = {}
no = 0


# Iterating through each frame
for i in range(len(frames)):

    # Getting the points in the world frame
    point_individual = point_cloud.get_segmented_points(frames[i], segments[i], poses[i], rotations[i])

    

    # Adding the list of points to the segmented point cloud list
    for key in point_individual.keys():

         # Ignoring floor and world body
        if key == -1 or key == 0:
            continue

        # Saving pure geoms
        if (m.geom(int(key)).bodyid[0] == 0):
            try:
                point_individual_pc[m.geom(int(key)).name].extend(point_individual[key])
                segmented_point_cloud[m.geom(int(key)).name].append(point_cloud.get_feature_seg_vector(m.geom(int(key)).name, np.array(point_individual[key])))
            except:
                point_individual_pc[m.geom(int(key)).name] = point_individual[key]
                segmented_point_cloud[m.geom(int(key)).name] = [point_cloud.get_feature_seg_vector(m.geom(int(key)).name, np.array(point_individual[key]))]
            #segmented_point_cloudlast[m.geom(int(key)).name].append(segmented_point_cloud[m.geom(int(key)).name])

            continue

        # Saving geoms that are part of a body
        body_id = m.geom(int(key)).bodyid[0]
        while m.body(body_id).parentid != 0:
            body_id = m.body(body_id).parentid[0]
        
        # Creating or extending the list of points for the body
        name = m.body(body_id).name
        try:
            point_individual_pc[name].extend(point_individual[key])
            segmented_point_cloud[name].append(point_cloud.get_feature_seg_vector(name, np.array(point_individual[key])))

        except:
            point_individual_pc[name] = point_individual[key]
            segmented_point_cloud[name] = [point_cloud.get_feature_seg_vector(name, np.array(point_individual[key]))]


    #segmented_point_cloudlast[i] = avg_feature_vector_seg(segmented_point_cloud)
    # Equivalent

    for key in segmented_point_cloud.keys():
        try:
            segmented_pc_f[key].append(segmented_point_cloud[key])
        except:
            segmented_pc_f[key] = segmented_point_cloud[key]
        #print(segmented_point_cloud[key])

#yes = avg_feature_vector_seg(segmented_pc_f)
x = []
y = []
z = []
amount = []
i = 0

for key in segmented_point_cloud.keys():
    print(len(segmented_point_cloud[key])) 
    for i in range(len(segmented_point_cloud[key])):
        print(segmented_pc_f[key][i][1])

                    
        x.append(segmented_point_cloud[key][i][0])
        y.append(segmented_point_cloud[key][i][1])
        z.append(segmented_point_cloud[key][i][2])
        amount.append(segmented_point_cloud[key][i][4])
    print("x ",x)
    np.array(x,axis=0)
        
    
    weig_avg_x = np.average(np.array(x,axis=0), weights=amount)
    weig_avg_y = np.average(np.array(y, axis=0), weights=amount)
    weig_avg_z = np.average(np.array(z, axis=0), weights=amount)
    segmented_pc_f[key] = [weig_avg_x, weig_avg_y, weig_avg_z, 0]


#segmented_point_cloudlast = segmented_pc_f
        
        #segmented_point_cloudlast[name].append(segmented_point_cloud[name])
    
                #if segmented_point_cloud != segmented_point_cloudlast:

        #print(segmented_point_cloud[key][i]) # x

        #print(point_individual_pc[key])


    
        #print(segmented_point_cloud)
        #print("")
        #no += 1
#print("num total ")
#print( no)
                    


7
-0.0015958300841654026
0.02311156944084847
[2.75703765e-01 2.31115694e-02 7.30736030e-02 0.00000000e+00
 2.74700000e+03]
-0.024582092169743414
[2.75703765e-01 2.31115694e-02 7.30736030e-02 0.00000000e+00
 2.74700000e+03]
0.024551618918207743
[2.75703765e-01 2.31115694e-02 7.30736030e-02 0.00000000e+00
 2.74700000e+03]
x  [0.26945596440562186, 0.2757037652589037, array([ 2.69455964e-01, -1.59583008e-03,  7.41941554e-02,  0.00000000e+00,
        2.80000000e+03]), 0.2843670948425084, array([ 2.69455964e-01, -1.59583008e-03,  7.41941554e-02,  0.00000000e+00,
        2.80000000e+03]), 0.28554725972521666, array([ 2.69455964e-01, -1.59583008e-03,  7.41941554e-02,  0.00000000e+00,
        2.80000000e+03])]


TypeError: array() got an unexpected keyword argument 'axis'

In [6]:
mat_avg

{'human3': [0.28554725972521666222,
  0.024551618918207743225,
  0.062579016732127915046,
  0],
 'human1': [0.25295066840433752281,
  -0.04413697665885053755,
  0.062085654686040292663,
  0],
 'human2': [0.21439014030696574716,
  0.12149407751381856749,
  0.06358339921748193947,
  0],
 'robot4': [0.21206947608012513461,
  0.16754169566551434908,
  0.06711712346740323488,
  0],
 'robot2': [0.056042168802897384273,
  0.19809008243048028245,
  0.08669887110140399247,
  0],
 'robot1': [0.03453067627480201516,
  0.13747441881679948477,
  0.09028268119602732741,
  0],
 'robot3': [0.038161438305026152587,
  0.12024284504522633189,
  0.09051505589117623681,
  0]}

In [15]:
segmented_point_cloud

{'human3': [array([ 2.69455964e-01, -1.59583008e-03,  7.41941554e-02,  0.00000000e+00,
          2.80000000e+03]),
  array([2.75703765e-01, 2.31115694e-02, 7.30736030e-02, 0.00000000e+00,
         2.74700000e+03]),
  [...],
  array([ 2.84367095e-01, -2.45820922e-02,  6.24788194e-02,  0.00000000e+00,
          7.85000000e+02]),
  [...],
  array([2.85547260e-01, 2.45516189e-02, 6.25790167e-02, 0.00000000e+00,
         7.86000000e+02]),
  [...]],
 'human1': [array([ 2.07773482e-01, -1.72729564e-01,  8.82477470e-02,  0.00000000e+00,
          4.76000000e+02]),
  [...],
  array([ 1.89577748e-01, -2.24444660e-01,  6.45143571e-02,  0.00000000e+00,
          1.92900000e+03]),
  [...],
  array([ 1.90914541e-01, -1.74861519e-01,  6.11467138e-02,  0.00000000e+00,
          4.13000000e+02]),
  [...]],
 'human2': [array([1.94955940e-01, 2.00211737e-01, 9.51587918e-02, 0.00000000e+00,
         9.92000000e+02]),
  array([1.89085463e-01, 1.74861614e-01, 6.11467030e-02, 0.00000000e+00,
         4.13000

In [9]:
mat_avg["human3"][1][0]

KeyError: 'human3'

In [16]:
key_list=list(mat_avg.keys())
key_list[0]

['human3', 'human1', 'human2', 'robot4', 'robot2', 'robot1', 'robot3']

In [1]:
m = mj.MjModel.from_xml_path('./mjcf/tec.xml')
d = mj.MjData(m)
r = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
r2 = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
r3 = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
r4 = mujoco.Renderer(m, 480, 640) # RGB = 1080 x 1920  default =  900, 1200   Depth =  640 x 480
opt = mujoco.MjvOption()
opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

frame = 0

cam = mujoco.MjvCamera()
result = {}
poses = []
rotations = []
points = []

point = []
point_cloud = []

# Add a dimension to the depth image to make it 3D
#                 depth = np.expand_dims(r_depth.render(), axis=2)
#                 # Limit the depth values to be between 0.3 and 4.0
#                 depth[depth < 0.3] = np.nan
#                 depth[depth > 4.0] = np.nan#

                # Add a dimension to the segmentation image to make it 3D
#                 seg = np.expand_dims(r_seg.render()[:, :, 0], axis=2)

                # Join the RGB and depth images
#                 rgb = r_rgb.render()
#                 result[cam] = np.append(rgb, depth, axis=2)
#                 result[cam] = np.append(result[cam], seg, axis=2)
t = 0
start_sim = d.time
no = -1
z_angle = 0

with mujoco.viewer.launch_passive(m, d) as viewer:
# Close the viewer automatically after 30 wall-seconds.
    with viewer.lock():
    
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)
    viewer.sync()
    
    
    
    start = time.time()
    while viewer.is_running() and time.time() - start < 5:
      step_start = time.time()
    
    
      # policy
      # mj_step can be replaced with code that also evaluates
      # a policy and applies a contro l signal before stepping the physics.
    
      #move_simple('human1', t=t, func=lambda x: (np.sin(x)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      #move_simple('human2', t=t, func=lambda x: (np.sin(x*2)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      #move_simple('human3', t=t, func=lambda x: (np.sin(x*3)+1)/2 * 0.2 + 0.05, axis=-1, model=m)
      mujoco.mj_step(m, d)
      t += 0.1
      r3.update_scene(d, camera="camera2")
      original = r3.render()
      #print(original.shape)
      
    
      # Pick up changes to the physics state, apply perturbations, update options from GUI.
      viewer.sync()
      r.enable_depth_rendering()
      r2.enable_depth_rendering()
      r4.enable_segmentation_rendering()
    
      r.update_scene(d, camera="camera",)
      r2.update_scene(d, camera="camera2",)
      r4.update_scene(d, camera="camera2")
      depth_r4 = r4.render() # Gives 480, 640 as shape
      
      geom_ids = depth_r4[:, :, 0]
      #print(geom_ids)
    
      #print(geom_ids.shape)
      
      depth_r2 = r2.render() # Gives 480, 640 as shape
      depth_r = r.render()
    
      #print(depth_r2)
      depth_r = np.expand_dims(depth_r, axis=2)
      depth = np.expand_dims(depth_r2, axis=2)
      depth[depth < 0.01] = np.nan
      depth[depth > 4.0] = np.nan
      #print(depth.shape)
      #print(depth_r.shape)
      #print(depth)
      
      
      
      rgb = r.render()
      no += 1
      result[no] = np.append(original, depth, axis=2)
      #print(result[no][:, :, 3].shape)
      pos = m.cam_pos[1] + m.body(m.cam_bodyid[1]).pos
      #print(pos)
      poses.append(pos)
      #angle = R.from_quat(m.body(m.cam_bodyid[1]).quat).as_euler('xyz', degrees=True)
      #rotations.append(angle[0])
      #print(result[no])
    
      #result = np.append(depth_r2, depth, axis=2)
      # To use as an image
      angle = R.from_quat(m.body(m.cam_bodyid[1]).quat).as_euler('xyz', degrees=True)
      rotations.append(angle[0])
      #print(angle[0])
    
      
      height = 480
      width = 640
      c_y = height / 2
      c_x = width / 2
      f_y = 0.5 * height / math.tan(60 * math.pi / 360)
      f_x = f_y * (width / height)
    
        # Expanding depth frame for matrix multiplication
      depth_frame = np.array(result[no][:, :, 3])
      depth_frame = np.expand_dims(depth_frame, axis=2)
      #print(depth_frame[0])


NameError: name 'mj' is not defined

In [63]:

pos = m.cam_pos[1] + m.body(m.cam_bodyid[1]).pos
print(pos)
poses.append(pos)

      #angle = R.from_quat(m.body(m.cam_bodyid[1]).quat).as_euler('xyz', degrees=True)
      #rotations.append(angle[0])
      #print(result[no])
    
      #result = np.append(depth_r2, depth, axis=2)
      # To use as an image
angle = R.from_quat(m.body(m.cam_bodyid[1]).quat).as_euler('xyz', degrees=True)
rotations.append(angle[0])
      #print(angle[0])

downsample = 5
index_y = np.linspace(0, height-1, height//downsample, dtype=int)
index_x = np.linspace(0, width-1, width//downsample, dtype=int)

# List of points in the world frame

# Creating rotation matrix
z_angle = -z_angle - 180  # Adjusting for the camera's orientation
rot = np.array([
                [math.cos(z_angle * np.pi / 180), -math.sin(z_angle * np.pi / 180), 0],
                [math.sin(z_angle * np.pi / 180), math.cos(z_angle * np.pi / 180), 0],
                [0, 0, 1]
                ])

[-0.01   0.2    0.205]


In [66]:
newPoint = []
for i in index_y:
    for j in index_x:
    
      D = depth_frame[i][j]
    
        # Skip if the pixel is not in the image
      if np.isnan(D[0]):
            continue
    
        # Obtaining world coordinates from depth pixels               
      mat = np.array([
                       [(i - c_y) / f_y],
                       [(j - c_x) * (width/height) / f_x],
                       [1]
                       ])
      point = np.matmul(mat, D)
        
        # Adjusting axis to match reconstruction
      transform = np.array([[0, 0, 1],
                              [0, -1, 0],
                              [-1, 0, 0]])
      point = np.matmul(transform, point)
    
        # Rotating the point to the world frame with pivot at the camera
      point = np.matmul(rot, point)
      


    
        # Translating the point to the world frame
      point = point + pos
        print
      

      #print(point)
      #newPoint = np.append(newPoint, point, axis=1)
      #print(point.shape)
    
    
    
      if point[2] < 0.03:
          continue
    
      # Adding the point to the list of points
      #points.append(point)
      points.append(point)


In [67]:
#mat = np.array(points)
print(len(point))

0


In [45]:
mat = np.array(points)



    

x = []
y = []
z = []
b = []
amount = []
weighted_avg = []
feature_vec_weigh_avg = {}

for key in mat_avg.keys():
    for i in range(len(mat_avg[key])):
        
        x.append(mat_avg[key][i][0])
        y.append(mat_avg[key][i][1])
        z.append(mat_avg[key][i][2])
        amount.append(mat_avg[key][i][4])

    weig_avg_x = np.average(x, weights=amount)
    weig_avg_y = np.average(y, weights=amount)
    weig_avg_z = np.average(z, weights=amount)
    feature_vec_weigh_avg[key] = [weig_avg_x, weig_avg_y, weig_avg_z, 0]

feature_vec_weigh_avg

        
    
    


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (13389,) + inhomogeneous part.